In [317]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('final_startup_dataset_v2.csv')

In [318]:
df.head()

,name,category_list,funding_total_usd,country_code,state_code,founded_at,funding_rounds,first_funding_at,last_funding_at,status
0,1 Mainstream,"['Apps', 'Cable', 'Distribution', 'Software']",5000000,USA,CA,2012-03-01,1,2015-03-17,2015-03-17,acquired
1,1000 Markets,"['Art', 'E-Commerce', 'Marketplaces']",500000,USA,WA,2009-01-01,1,2009-05-15,2009-05-15,acquired
2,1000memories,['Curated Web'],2535000,USA,CA,2010-07-01,2,2010-01-01,2011-02-16,acquired
3,100Plus,['Analytics'],1250000,USA,CA,2011-09-16,2,2011-11-02,2011-11-30,acquired
4,1010data,['Software'],35000000,USA,NY,2000-01-01,1,2010-03-08,2010-03-08,acquired


In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13959 entries, 0 to 13958
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               13958 non-null  object
 1   category_list      12873 non-null  object
 2   funding_total_usd  13959 non-null  object
 3   country_code       11968 non-null  object
 4   state_code         11750 non-null  object
 5   founded_at         10227 non-null  object
 6   funding_rounds     13959 non-null  int64 
 7   first_funding_at   13957 non-null  object
 8   last_funding_at    13959 non-null  object
 9   status             13959 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.1+ MB


In [320]:
df.isnull().sum()

name                    1
category_list        1086
funding_total_usd       0
country_code         1991
state_code           2209
founded_at           3732
funding_rounds          0
first_funding_at        2
last_funding_at         0
status                  0
dtype: int64

In [321]:
def status_to_binary(status):
    if status in ['acquired', 'ipo']:
        return 1
    elif status == 'closed':
        return 0
    else:
        return None  # if any unexpected value

# Apply to new column
df['status_binary'] = df['status'].apply(status_to_binary)

In [322]:
missing_values = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100
}).sort_values(by='Missing Count', ascending=False)

print(missing_values)

                   Missing Count  Missing %
founded_at                  3732  26.735440
state_code                  2209  15.824916
country_code                1991  14.263199
category_list               1086   7.779927
first_funding_at               2   0.014328
name                           1   0.007164
funding_total_usd              0   0.000000
funding_rounds                 0   0.000000
last_funding_at                0   0.000000
status                         0   0.000000
status_binary                  0   0.000000


In [323]:
df['founded_at'] = pd.to_datetime(df['founded_at'], errors='coerce')
df['first_funding_at'] = pd.to_datetime(df['first_funding_at'], errors='coerce')
df['last_funding_at'] = pd.to_datetime(df['last_funding_at'], errors='coerce')


In [324]:
df.head()

,name,category_list,funding_total_usd,country_code,state_code,founded_at,funding_rounds,first_funding_at,last_funding_at,status,status_binary
0,1 Mainstream,"['Apps', 'Cable', 'Distribution', 'Software']",5000000,USA,CA,2012-03-01,1,2015-03-17,2015-03-17,acquired,1
1,1000 Markets,"['Art', 'E-Commerce', 'Marketplaces']",500000,USA,WA,2009-01-01,1,2009-05-15,2009-05-15,acquired,1
2,1000memories,['Curated Web'],2535000,USA,CA,2010-07-01,2,2010-01-01,2011-02-16,acquired,1
3,100Plus,['Analytics'],1250000,USA,CA,2011-09-16,2,2011-11-02,2011-11-30,acquired,1
4,1010data,['Software'],35000000,USA,NY,2000-01-01,1,2010-03-08,2010-03-08,acquired,1


In [325]:
df = df.drop(columns=['status'])

In [326]:
df.dropna(subset=['first_funding_at'], inplace=True)

In [327]:
missing_values = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100
}).sort_values(by='Missing Count', ascending=False)

print(missing_values)

                   Missing Count  Missing %
founded_at                  3734  26.753600
state_code                  2209  15.827183
country_code                1991  14.265243
category_list               1086   7.781042
name                           1   0.007165
funding_total_usd              0   0.000000
funding_rounds                 0   0.000000
first_funding_at               0   0.000000
last_funding_at                0   0.000000
status_binary                  0   0.000000


In [328]:
df['state_code'] = df['state_code'].fillna('unknown')


In [329]:
# First, create a mapping of the most common country_code for each state_code
state_to_country = (
    df[df['country_code'].notna() & df['state_code'].notna()]
    .groupby('state_code')['country_code']
    .agg(lambda x: x.value_counts().index[0])
)

# Now, fill missing country_code based on that mapping
def fill_country_code(row):
    if pd.isna(row['country_code']):
        return state_to_country.get(row['state_code'], np.nan)
    return row['country_code']

df['country_code'] = df.apply(fill_country_code, axis=1)

In [330]:
missing_values = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100
}).sort_values(by='Missing Count', ascending=False)

print(missing_values)

                   Missing Count  Missing %
founded_at                  3734  26.753600
category_list               1086   7.781042
name                           1   0.007165
funding_total_usd              0   0.000000
country_code                   0   0.000000
state_code                     0   0.000000
funding_rounds                 0   0.000000
first_funding_at               0   0.000000
last_funding_at                0   0.000000
status_binary                  0   0.000000


In [331]:
df.dropna(subset=['category_list'], inplace=True)

In [332]:
import pandas as pd

# Ensure datetime format
df['first_funding_at'] = pd.to_datetime(df['first_funding_at'], errors='coerce')
df['founded_at'] = pd.to_datetime(df['founded_at'], errors='coerce')

# Calculate mean time difference where both dates are available
valid_dates = df[df['first_funding_at'].notna() & df['founded_at'].notna()]
mean_diff = (valid_dates['first_funding_at'] - valid_dates['founded_at']).mean()

# Fill missing founded_at as first_funding_at - mean_diff
df.loc[df['founded_at'].isna(), 'founded_at'] = df['first_funding_at'] - mean_diff


In [333]:
missing_values = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100
}).sort_values(by='Missing Count', ascending=False)

print(missing_values)

                   Missing Count  Missing %
name                           1   0.007769
category_list                  0   0.000000
funding_total_usd              0   0.000000
country_code                   0   0.000000
state_code                     0   0.000000
founded_at                     0   0.000000
funding_rounds                 0   0.000000
first_funding_at               0   0.000000
last_funding_at                0   0.000000
status_binary                  0   0.000000


In [334]:
df['name'] = df['name'].fillna('unknown')


In [335]:
df = df[df['name'].notnull()]

In [336]:
missing_values = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100
}).sort_values(by='Missing Count', ascending=False)

print(missing_values)

                   Missing Count  Missing %
name                           0        0.0
category_list                  0        0.0
funding_total_usd              0        0.0
country_code                   0        0.0
state_code                     0        0.0
founded_at                     0        0.0
funding_rounds                 0        0.0
first_funding_at               0        0.0
last_funding_at                0        0.0
status_binary                  0        0.0


In [337]:
df.shape

(12871, 10)

In [338]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12871 entries, 0 to 13958
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               12871 non-null  object        
 1   category_list      12871 non-null  object        
 2   funding_total_usd  12871 non-null  object        
 3   country_code       12871 non-null  object        
 4   state_code         12871 non-null  object        
 5   founded_at         12871 non-null  datetime64[ns]
 6   funding_rounds     12871 non-null  int64         
 7   first_funding_at   12871 non-null  datetime64[ns]
 8   last_funding_at    12871 non-null  datetime64[ns]
 9   status_binary      12871 non-null  int64         
dtypes: datetime64[ns](3), int64(2), object(5)
memory usage: 1.6+ MB


In [339]:
(df['funding_total_usd'] == '-').sum()

1512

In [340]:
# Step 1: Replace '-' with NaN
df['funding_total_usd'] = df['funding_total_usd'].replace('-', np.nan)

# Step 2: Convert to numeric
df['funding_total_usd'] = pd.to_numeric(df['funding_total_usd'], errors='coerce')

# Step 3: Fill NaN with mean funding per country
df['funding_total_usd'] = df.groupby('country_code')['funding_total_usd'] \
                            .transform(lambda x: x.fillna(x.mean()))

In [341]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12871 entries, 0 to 13958
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               12871 non-null  object        
 1   category_list      12871 non-null  object        
 2   funding_total_usd  12866 non-null  float64       
 3   country_code       12871 non-null  object        
 4   state_code         12871 non-null  object        
 5   founded_at         12871 non-null  datetime64[ns]
 6   funding_rounds     12871 non-null  int64         
 7   first_funding_at   12871 non-null  datetime64[ns]
 8   last_funding_at    12871 non-null  datetime64[ns]
 9   status_binary      12871 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(2), object(4)
memory usage: 1.6+ MB


In [342]:
df.to_csv('final_startup_dataset_v4.csv', index=False)